# Machine Learning Model

Machine Learning Model for user sign in - this takes in a list of keystroke patterns of the attempted logins from the user. It trains the model each attempt and returns a confidence score based on the new provided keystroke pattern vs what the system would expect.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Data Preperation
#data = #This will be the list sent over from the C# project
#expected = #This will be the bit value from the table
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data)

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)
